<a href="https://colab.research.google.com/github/around-star/Emojify/blob/master/Emojify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##EMOJIFY

In [1]:
import os
from google.colab import drive
drive.mount('/gdrive')
os.chdir('/gdrive/My Drive/Colab Notebooks')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [2]:
!pip install import-ipynb
import import_ipynb

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=10b9fdaf326658492153211f32912f61d4f5d9df40b94331ba3eb8f9d0371429
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb


In [3]:
import emo 
import numpy as np
import emoji

importing Jupyter notebook from emo.ipynb
     |████████████████████████████████| 51kB 2.9MB/s 
  Created wheel for emoji: filename=emoji-0.5.4-cp36-none-any.whl size=42176 sha256=e6ead79a0be8ac33162d50e2348d2f155d122c91f8eb0d359767214636074fc7
  Stored in directory: /root/.cache/pip/wheels/2a/a9/0a/4f8e8cce8074232aba240caca3fade315bb49fac68808d1a9c
Successfully built emoji


In [29]:
os.chdir('/gdrive/My Drive')
X_train , Y_train = emo.read_csv('data/train_emoji.csv')
X_test, Y_test = emo.read_csv('data/test.csv')

In [5]:
max_len = len(max(X_train, key=len).split())

In [6]:
for i in range(10):
  print(X_train[i], emo.label_to_emoji(Y_train[i]))

never talk to me again 😞
I am proud of your achievements 😄
It is the worst day in my life 😞
Miss you so much ❤️
food is life 🍴
I love you mum ❤️
Stop saying bullshit 😞
congratulations on your acceptance 😄
The assignment is too long  😞
I want to go play ⚾


In [7]:
def one_hot_encode(Y, num = 5):
  n = len(Y)
  y = np.zeros((n,num))
  y[range(n), Y] = 1
  return y

In [8]:
Y_test[:10]

array([4, 3, 2, 2, 2, 2, 3, 2, 4, 2])

In [9]:
word_to_index, index_to_word, word_to_vec_map = emo.read_glove_vecs('data/glove.6B.50d.txt')

In [11]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Activation, LSTM, Embedding
from tensorflow.keras.initializers import glorot_uniform

In [12]:
def sentences_to_indices(X, word_to_index, max_len):
  m = X.shape[0]

  x_indices = np.zeros((m, max_len))

  for i in range(m):

    sentence_words = X[i].lower().split()
    j=0
    
    for w in sentence_words:

      x_indices[i,j] = word_to_index[w]
      j+=1

  return x_indices

In [13]:
def pretrained_embedded_layer(word_to_vec_map, word_to_index):

  vocab_len = len(word_to_index) + 1
  emb_dim = 50

  emb_matrix = np.zeros((vocab_len, emb_dim))

  for word,i in word_to_index.items():
    emb_matrix[i,:] = word_to_vec_map[word]

  embedding_layer = Embedding(vocab_len, emb_dim, trainable = False)
  embedding_layer.build((None,))
  embedding_layer.set_weights([emb_matrix])

  return embedding_layer

In [14]:
def model(input_shape, word_to_vec_map, word_to_index):

  sentence_indices = Input(shape = input_shape)

  embedding_layer = pretrained_embedded_layer(word_to_vec_map, word_to_index)

  embeddings = embedding_layer(sentence_indices)

  X = LSTM(128, return_sequences=True)(embeddings)
  X = Dropout(0.5)(X)
  X = LSTM(128)(X)
  X = Dropout(0.5)(X)
  X = Dense(5)(X)
  X = Activation('softmax')(X)

  model = Model(inputs = sentence_indices, outputs = X)

  return model

In [15]:
model = model(max_len, word_to_vec_map, word_to_index)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 10)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 10, 50)            20000050  
_________________________________________________________________
lstm (LSTM)                  (None, 10, 128)           91648     
_________________________________________________________________
dropout (Dropout)            (None, 10, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 5)                 645   

In [16]:
model.compile(loss= 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [17]:
X_train = sentences_to_indices(X_train, word_to_index, max_len)
Y_train = one_hot_encode(Y_train)

In [18]:
model.fit(X_train, Y_train, epochs=50, batch_size=32)

Epoch 1/50
5/5 [==============================] - 0s 11ms/step - loss: 1.5920 - accuracy: 0.2348
Epoch 2/50
5/5 [==============================] - 0s 8ms/step - loss: 1.5040 - accuracy: 0.3636
Epoch 3/50
5/5 [==============================] - 0s 9ms/step - loss: 1.4747 - accuracy: 0.3030
Epoch 4/50
5/5 [==============================] - 0s 9ms/step - loss: 1.4087 - accuracy: 0.3485
Epoch 5/50
5/5 [==============================] - 0s 9ms/step - loss: 1.3190 - accuracy: 0.5606
Epoch 6/50
5/5 [==============================] - 0s 8ms/step - loss: 1.2540 - accuracy: 0.4697
Epoch 7/50
5/5 [==============================] - 0s 9ms/step - loss: 1.1342 - accuracy: 0.5303
Epoch 8/50
5/5 [==============================] - 0s 8ms/step - loss: 1.0046 - accuracy: 0.6288
Epoch 9/50
5/5 [==============================] - 0s 9ms/step - loss: 0.9466 - accuracy: 0.6288
Epoch 10/50
5/5 [==============================] - 0s 9ms/step - loss: 0.7950 - accuracy: 0.6818
Epoch 11/50
5/5 [=====================

In [19]:
X_test = sentences_to_indices(X_test, word_to_index, max_len)
Y_test = one_hot_encode(Y_test)
loss, acc = model.evaluate(X_test, Y_test)
print()
print("accuracy is ", acc)

2/2 [==============================] - 0s 5ms/step - loss: 0.4895 - accuracy: 0.8750

accuracy is  0.875


##TEST

In [41]:
X = np.array(['I am hungry','This has been a pretty good day so long.'])
X_indices = sentences_to_indices(X, word_to_index, max_len)
pred = model.predict(X_indices)
for i in range(len(X)):
  print(X[i], '', emo.label_to_emoji(np.argmax(pred[i])))

I am hungry  🍴
This has been a pretty good day so long.  😄
